# SOLUTION: Data Exploration and Feature Engineering

## Imports

In [1]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F

import json
import os

## Create Snowpark Session

In [2]:
os.chdir('/Users/nicholasaverillo/Documents/MachineLearning/sfguide-getting-started-machine-learning/hol/')
print(os.getcwd())
with open('creds.json') as f:
    connection_parameters = json.load(f)

/Users/nicholasaverillo/Documents/MachineLearning/sfguide-getting-started-machine-learning/hol


In [3]:
session = Session.builder.configs(connection_parameters).create()
print(f"Current Database and schema: {session.get_fully_qualified_current_schema()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")

Current Database and schema: "HOL_DB"."PUBLIC"
Current Warehouse: "HOL_WH"


In [4]:
snowpark_df = session.table('APPLICATION_RECORD')

In [5]:
snowpark_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"     |"CODE_GENDER"  |"FLAG_OWN_CAR"  |"FLAG_OWN_REALTY"  |"CNT_CHILDREN"  |"AMT_INCOME_TOTAL"  |"NAME_INCOME_TYPE"    |"NAME_EDUCATION_TYPE"          |"NAME_FAMILY_STATUS"  |"NAME_HOUSING_TYPE"  |"DAYS_BIRTH"  |"DAYS_EMPLOYED"  |"FLAG_MOBIL"  |"FLAG_WORK_PHONE"  |"FLAG_PHONE"  |"FLAG_EMAIL"  |"OCCUPATION_TYPE"  |"CNT_FAM_MEMBERS"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# SOLUTION: Answer the following Questions

In [6]:
# How many variables (columns) does the dataset have?
len(snowpark_df.columns)

18

In [7]:
# How many rows does the dataset have?
snowpark_df.count()

438557

In [8]:
# What is the average income?
snowpark_df.select(F.avg('AMT_INCOME_TOTAL').as_('AVERAGE_INCOME')).show()

---------------------
|"AVERAGE_INCOME"   |
---------------------
|187524.2860095039  |
---------------------



In [9]:
# How many people are Single?
snowpark_df.filter(F.col('NAME_FAMILY_STATUS') == 'Single / not married').count()

55271

# SOLUTION: Create / Drop Features

In [10]:
# Create a new feature containing the years of employment
# Formula: Absolute Value of DAYS_EMPLOYED divided by 365 days rounded down
snowpark_df = snowpark_df.with_column('WORKYEARS', F.floor(F.abs(F.col('DAYS_EMPLOYED')) / 365))
snowpark_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"     |"CODE_GENDER"  |"FLAG_OWN_CAR"  |"FLAG_OWN_REALTY"  |"CNT_CHILDREN"  |"AMT_INCOME_TOTAL"  |"NAME_INCOME_TYPE"    |"NAME_EDUCATION_TYPE"          |"NAME_FAMILY_STATUS"  |"NAME_HOUSING_TYPE"  |"DAYS_BIRTH"  |"DAYS_EMPLOYED"  |"FLAG_MOBIL"  |"FLAG_WORK_PHONE"  |"FLAG_PHONE"  |"FLAG_EMAIL"  |"OCCUPATION_TYPE"  |"CNT_FAM_MEMBERS"  |"WORKYEARS"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
# Drop the variable DAYS_EMPLOYED
snowpark_df = snowpark_df.drop('DAYS_EMPLOYED')
snowpark_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"     |"CODE_GENDER"  |"FLAG_OWN_CAR"  |"FLAG_OWN_REALTY"  |"CNT_CHILDREN"  |"AMT_INCOME_TOTAL"  |"NAME_INCOME_TYPE"    |"NAME_EDUCATION_TYPE"          |"NAME_FAMILY_STATUS"  |"NAME_HOUSING_TYPE"  |"DAYS_BIRTH"  |"FLAG_MOBIL"  |"FLAG_WORK_PHONE"  |"FLAG_PHONE"  |"FLAG_EMAIL"  |"OCCUPATION_TYPE"  |"CNT_FAM_MEMBERS"  |"WORKYEARS"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
session.close()